In [5]:
import pandas as pd

# datasets
birthrate_df = pd.read_csv("formatted_birth_data.csv")
workhours_df = pd.read_csv("eurostat_workhours_fully_cleaned .csv")
expenditure_df = pd.read_csv("eu_family_expenditure_full.csv")
price_index_df = pd.read_csv("eu_comparative_price_levels_2015_2023.csv")

# total population work hours 
work_hours_filtered = workhours_df[
    (workhours_df['sex'] == 'total') & (workhours_df['age'] == '15__64')
][['geo', 'time', 'employees_total']].copy()
work_hours_filtered.rename(columns={
    'geo': 'country_name',
    'time': 'year',
    'employees_total': 'weekly_hours'
}, inplace=True)

# map country names
country_name = {
    'austria': 'AT', 'belgium': 'BE', 'bulgaria': 'BG', 'croatia': 'HR', 'cyprus': 'CY',
    'czechia': 'CZ', 'germany': 'DE', 'denmark': 'DK', 'estonia': 'EE', 'greece': 'EL',
    'spain': 'ES', 'finland': 'FI', 'france': 'FR', 'hungary': 'HU', 'ireland': 'IE',
    'italy': 'IT', 'lithuania': 'LT', 'luxembourg': 'LU', 'latvia': 'LV', 'malta': 'MT',
    'netherlands': 'NL', 'poland': 'PL', 'portugal': 'PT', 'romania': 'RO', 'sweden': 'SE',
    'slovenia': 'SI', 'slovakia': 'SK'
}
work_hours_filtered['country_code'] = work_hours_filtered['country_name'].str.lower().map(country_name)
work_hours_filtered.dropna(subset=['country_code'], inplace=True)

# expenditure data
selected_benefits = ['Cash benefits', 'Parental leave (periodic)', 'Childcare services']
expenditure_subset = expenditure_df[
    (expenditure_df['benefit_type'].isin(selected_benefits)) &
    (expenditure_df['unit'] == 'PPS per person')
].copy()
expenditure_grouped = expenditure_subset.groupby(['country', 'year'])['expenditure'].sum().reset_index()
expenditure_grouped.rename(columns={
    'country': 'country_code',
    'expenditure': 'expenditure_per_capita'
}, inplace=True)

# price index data
price_index_long = price_index_df.melt(id_vars='Country', var_name='year', value_name='price_index')
price_index_long['year'] = price_index_long['year'].astype(int)
country_name_caps = {k.title(): v for k, v in country_name_to_iso2.items()}
price_index_long['country_code'] = price_index_long['Country'].map(country_name_caps)
price_index_long.dropna(subset=['country_code'], inplace=True)

# Merge everything 
merged = birthrate_df.merge(
    work_hours_filtered, left_on=['country', 'year'], right_on=['country_code', 'year']
).merge(
    expenditure_grouped, on=['country_code', 'year']
).merge(
    price_index_long[['country_code', 'year', 'price_index']], on=['country_code', 'year']
)

# setup
final_model_df = merged[[
    'country_code', 'year',
    'weekly_hours', 'expenditure_per_capita',
    'price_index', 'birth_rate_per_thousand'
]].dropna()

# save 
final_model_df.to_csv("final_merged.csv", index=False)



In [ ]:

main_df = pd.read_csv("final_merged.csv")
raw_expend = pd.read_csv("eu_family_expenditure_full.csv")


# Filter Rows

filtered = raw_expend[
    (raw_expend['scheme'] == 'TOTAL') &
    (raw_expend['unit'].str.contains("per person", case=False, na=False))
]


# Benefit Types

cash_types = [
    'Cash benefits', 'Lump-sum cash', 'Birth grants', 'Other lump-sum benefits',
    'Cash periodic', 'Family allowance', 'Maternity/childbirth benefits',
    'Other cash periodic'
]

leave_types = [
    'Parental leave (periodic)', 'Parental leave (lump-sum)', 'Maternity/childbirth benefits'
]

service_types = [
    'Childcare services', 'Home-based support'
]

def categorize_benefit(bt):
    if bt in cash_types:
        return 'cash_per_capita'
    elif bt in leave_types:
        return 'leave_per_capita'
    elif bt in service_types:
        return 'services_per_capita'
    else:
        return None

filtered = filtered.copy()
filtered['benefit_category'] = filtered['benefit_type'].map(categorize_benefit)
filtered = filtered.dropna(subset=['benefit_category'])


# Pivot

pivot_df = filtered.groupby(['country', 'year', 'benefit_category'])['expenditure'].mean().unstack().reset_index()
pivot_df.rename(columns={'country': 'country_code'}, inplace=True)


# Merge with main

pivot_df['year'] = pivot_df['year'].astype(int)
main_df['year'] = main_df['year'].astype(int)
merged_df = pd.merge(main_df, pivot_df, on=['country_code', 'year'], how='inner')


#  Missing Values
merged_df.dropna(subset=['cash_per_capita', 'leave_per_capita', 'services_per_capita'], inplace=True)

# Save 

merged_df.to_csv("final_cleaned.csv", index=False)
